In [3]:
import pandas as pd
data = pd.read_csv('payment_data.csv')

In [9]:
data.columns

Index(['user_id', 'date', 'sum_of_deposit', 'sum_of_withdraw'], dtype='object')

In [4]:
data.shape

(290426, 4)

In [6]:
data[((data['sum_of_deposit']!=0) &
      (data['sum_of_withdraw']!=0))]

,user_id,date,sum_of_deposit,sum_of_withdraw
5,1,2024-01-06,210.02,100.85
15,1,2024-01-16,152.16,65.59
16,1,2024-01-17,104.18,139.39
23,1,2024-01-24,125.61,121.03
26,1,2024-01-28,149.70,157.67
...,...,...,...,...
290413,1000,2024-12-13,255.49,267.46
290415,1000,2024-12-17,90.73,393.10
290416,1000,2024-12-18,122.26,327.68
290418,1000,2024-12-20,98.50,257.72


# Models

In [ ]:
def moving_average(df, window=3):
    """Computes a moving average for the monthly difference."""
    df['date'] = pd.to_datetime(df[['year', 'month']].assign(day=1))
    df = df.sort_values('date')
    df['moving_avg'] = df['difference'].rolling(window=window, min_periods=1).mean()
    return df[['year', 'month', 'difference', 'moving_avg']]


def monthly_difference(df):
    """Calculates monthly differences in 'red' values."""
    # Убедимся, что 'date' в нужном формате
    df['date'] = pd.to_datetime(df['date'])

    # Получаем максимальное значение 'red' по каждому месяцу
    monthly_max = df.groupby(df['date'].dt.to_period('M'))['red'].max()

    # Преобразуем в DataFrame и сбросим индекс
    monthly_max = monthly_max.reset_index()
    monthly_max['date'] = monthly_max['date'].dt.to_timestamp()

    # Вычисляем разницу с предыдущим месяцем
    monthly_max['difference'] = monthly_max['red'].diff().fillna(monthly_max['red'])

    # Добавим год и месяц как отдельные колонки (если нужно)
    monthly_max['year'] = monthly_max['date'].dt.year
    monthly_max['month'] = monthly_max['date'].dt.month

    # Результат
    result = monthly_max[['date', 'year', 'month', 'red', 'difference']]
    return result[['year', 'month', 'difference']]

def predict_player_full_info(data_one_user):
    """Calculates numerical player value metrics."""
    result_df = monthly_difference(data_one_user)
    #print('results_df', result_df)
    moving_avg_df = moving_average(result_df)
    #print('Moving average',moving_avg_df)
    mean_pv = moving_avg_df['moving_avg'].mean()
    std_pv = moving_avg_df['moving_avg'].std()
    threshold = mean_pv + 1.28 * std_pv
    filtered = moving_avg_df[moving_avg_df['moving_avg'] <= threshold]
    pv0_90 = filtered['moving_avg'].max()
    pv0_90_b = threshold
    pv0_max = moving_avg_df['moving_avg'].max()
    dpv = float(data_one_user['red'].iloc[-1] - data_one_user['blue'].iloc[-1])
    #pv_num = pv0 + dpv
    #return {"pv": pv_num, "pv0_90": pv0_90, "pv0_90_b": pv0_90_b, "dpv": dpv}
    return {"pv0_90": pv0_90, "pv0_90_b": pv0_90_b, "pv0_max": pv0_max}

def get_red_line_for_true_model_net_dep(data_one_user):
    # Сортировка и расчёт blue
    data_one_user = data_one_user.sort_values(by='d').reset_index(drop=True)

    # Копируем первую строку
    fake_row = data_one_user.iloc[[0]].copy()

    # Заменяем нужные поля
    fake_date = data_one_user['d'].min() - pd.Timedelta(days=1)
    fake_row['d'] = fake_date
    fake_row['sum_of_deposits'] = 0
    fake_row['sum_of_withdrawals'] = 0

    # Объединяем и пересчитываем red
    data_one_user = pd.concat([fake_row, data_one_user], ignore_index=True)


    data_one_user['net_dep'] = data_one_user['sum_of_deposits'] - data_one_user['sum_of_withdrawals']
    data_one_user['blue'] = data_one_user['net_dep'].cumsum()
    data_one_user['red'] = data_one_user['blue'].cummax()
    data_one_user['date'] = pd.to_datetime(data_one_user['d'])

    return data_one_user


def get_red_line_exp_dec(data_one_user):
    # Сортировка по дате
    data_one_user = data_one_user.sort_values(by='d').reset_index(drop=True)

    # Копируем первую строку
    fake_row = data_one_user.iloc[[0]].copy()

    # Заменяем нужные поля
    fake_date = data_one_user['d'].min() - pd.Timedelta(days=1)
    fake_row['d'] = fake_date
    fake_row['sum_of_deposits'] = 0
    fake_row['sum_of_withdrawals'] = 0

    # Объединяем и пересчитываем red
    data_one_user = pd.concat([fake_row, data_one_user], ignore_index=True)

    df = data_one_user.sort_values('d').reset_index(drop=True)

    # Расчёт net_dep
    df['net_dep'] = df['sum_of_deposits'] - df['sum_of_withdrawals']
    df['net_dep_cum_sum'] = df['net_dep'].cumsum()

    # Преобразуем даты в массив
    dates = df['d'].values.astype('datetime64[D]')
    date_diff_matrix = (dates[:, None] - dates[None, :]).astype('timedelta64[D]').astype(int)

    # Мы хотим только те, где i <= j (прошлое к текущему)
    mask = date_diff_matrix >= 0

    # Параметр затухания
    tau = 60

    # Вычисляем веса затухания
    decay_weights = np.exp(-date_diff_matrix / tau) * mask

    # Вектор net_dep
    net_dep_vector = df['net_dep'].values

    # Вычисляем экспоненциально-взвешенную сумму
    net_dep_cum_exp = decay_weights @ net_dep_vector

    # Добавляем в датафрейм
    df['net_dep_cum_exp'] = net_dep_cum_exp
    df['blue'] = df['net_dep_cum_exp']
    df['red'] = df['blue'].cummax()
    data_one_user = df
    data_one_user['date'] = pd.to_datetime(data_one_user['d'])
    return data_one_user

def get_red_line_adapt_exp_dec(data_one_user):
    # Сортировка по дате
    data_one_user = data_one_user.sort_values(by='d').reset_index(drop=True)

    # Копируем первую строку
    fake_row = data_one_user.iloc[[0]].copy()

    # Заменяем нужные поля
    fake_date = data_one_user['d'].min() - pd.Timedelta(days=1)
    fake_row['d'] = fake_date
    fake_row['sum_of_deposits'] = 0
    fake_row['sum_of_withdrawals'] = 0

    # Объединяем и пересчитываем red
    data_one_user = pd.concat([fake_row, data_one_user], ignore_index=True)

    df = data_one_user.sort_values('d').reset_index(drop=True)

    # Расчёт net_dep
    df['net_dep'] = df['sum_of_deposits'] - df['sum_of_withdrawals']
    df['net_dep_cum_sum'] = df['net_dep'].cumsum()

    # Преобразуем даты в массив
    dates = df['d'].values.astype('datetime64[D]')
    date_diff_matrix = (dates[:, None] - dates[None, :]).astype('timedelta64[D]').astype(int)

    # Мы хотим только те, где i <= j (прошлое к текущему)
    mask = date_diff_matrix >= 0

    # Параметр затухания
    tau = 60

    # Вычисляем веса затухания
    decay_weights = np.exp(-date_diff_matrix / tau) * mask

    # Вектор net_dep
    net_dep_vector = df['net_dep'].values
    dep_vector = df['sum_of_deposits'].cumsum().values
    #withd_vector = df['sum_of_withdrawals'].cumsum().values
    #withd_vector = df['sum_of_withdrawals'].cumsum().values
    withd_vector = (decay_weights * df['sum_of_withdrawals'].values[None, :]).sum(axis=1)

    # Вычисляем экспоненциально-взвешенную сумму
    #net_dep_cum_exp = dep_vector - (withd_vector @ decay_weights)
    net_dep_cum_exp = dep_vector - withd_vector

    # Добавляем в датафрейм
    df['net_dep_cum_exp'] = net_dep_cum_exp
    df['blue'] = df['net_dep_cum_exp']
    df['red'] = df['blue'].cummax()
    data_one_user = df
    data_one_user['date'] = pd.to_datetime(data_one_user['d'])
    return df


def predict_pv_exp_full(data_one_user):
    
    data_one_user = get_red_line_exp_dec(data_one_user)
    result = predict_player_full_info(data_one_user)
    
    return result

def predict_pv_net_dep_true(data_one_user):
    
    data_one_user = get_red_line_for_true_model_net_dep(data_one_user)
    result = predict_player_full_info(data_one_user)
    
    return result

In [31]:
import numpy as np

100*np.exp(-1*30/60)

np.float64(60.653065971263345)

In [32]:
(3*1000+300+3)/9

367.0